In [14]:
!pip install scikit-surprise


Defaulting to user installation because normal site-packages is not writeable
  Using cached scikit_surprise-1.1.4.tar.gz (154 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build scikit-surprise


  error: subprocess-exited-with-error
  
  × Building wheel for scikit-surprise (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [115 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build\lib.win-amd64-cpython-312\surprise
      copying surprise\accuracy.py -> build\lib.win-amd64-cpython-312\surprise
      copying surprise\builtin_datasets.py -> build\lib.win-amd64-cpython-312\surprise
      copying surprise\dataset.py -> build\lib.win-amd64-cpython-312\surprise
      copying surprise\dump.py -> build\lib.win-amd64-cpython-312\surprise
      copying surprise\reader.py -> build\lib.win-amd64-cpython-312\surprise
      copying surprise\trainset.py -> build\lib.win-amd64-cpython-312\surprise
      copying surprise\utils.py -> build\lib.win-amd64-cpython-312\surprise
      copying surprise\__init__.py -> build\lib.win-amd64-cpython-312\surprise
      copying surprise\__main__.py -> build\lib.win-amd64-cpython-312\surpris

In [15]:
import numpy as np
import pandas as pd

# Step 1: Load the dataset (you would typically use pd.read_csv to load a CSV file,
# but since you provided it in text form, we'll assume it's already loaded into a DataFrame).

# Example DataFrame
data = pd.read_csv(r'C:\Users\anura\Downloads\RS DATASET\RS Practical 5\Womens Clothing E-Commerce Reviews.csv')

data.dropna(inplace=True)
data.drop(['Unnamed: 0'], axis=1, inplace=True)
data.head()

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses
5,1080,49,Not for the very petite,"I love tracy reese dresses, but this one is no...",2,0,4,General,Dresses,Dresses
6,858,39,Cagrcoal shimmer fun,I aded this in my basket at hte last mintue to...,5,1,1,General Petite,Tops,Knits


In [16]:

# Step 2: Create a user-item interaction matrix
# We will assume each user is uniquely represented by their 'Age' and Clothing ID as items.

# Assign each unique clothing item to a column
clothing_items = df['Clothing ID'].unique()
users = df['Age'].unique()

# Create a mapping from clothing items to indices
item_index = {item: idx for idx, item in enumerate(clothing_items)}
user_index = {user: idx for idx, user in enumerate(users)}

# Initialize the user-item interaction matrix (ratings matrix)
user_item_matrix = np.zeros((len(users), len(clothing_items)))

# Populate the user-item matrix with ratings
for _, row in df.iterrows():
    user_idx = user_index[row['Age']]
    item_idx = item_index[row['Clothing ID']]
    user_item_matrix[user_idx, item_idx] = row['Rating']

# Step 3: Matrix factorization using gradient descent
num_users, num_items = user_item_matrix.shape
num_factors = 2  # Latent factors
alpha = 0.01     # Learning rate
num_epochs = 100
lambda_reg = 0.02  # Regularization parameter

# Initialize user and item latent factor matrices with random values
user_factors = np.random.rand(num_users, num_factors)
item_factors = np.random.rand(num_items, num_factors)

# Step 4: Perform matrix factorization using Gradient Descent
for epoch in range(num_epochs):
    for i in range(num_users):
        for j in range(num_items):
            if user_item_matrix[i, j] > 0:  # Only update for existing ratings
                # Calculate the prediction error for user i and item j
                prediction = np.dot(user_factors[i, :], item_factors[j, :].T)
                error = user_item_matrix[i, j] - prediction

                # Update user and item latent factors
                user_factors[i, :] += alpha * (error * item_factors[j, :] - lambda_reg * user_factors[i, :])
                item_factors[j, :] += alpha * (error * user_factors[i, :] - lambda_reg * item_factors[j, :])

    # Calculate the total error every 1000 epochs for monitoring
    if epoch % 10 == 0:
        total_error = 0
        for i in range(num_users):
            for j in range(num_items):
                if user_item_matrix[i, j] > 0:
                    prediction = np.dot(user_factors[i, :], item_factors[j, :].T)
                    total_error += (user_item_matrix[i, j] - prediction) ** 2
        print(f"Epoch {epoch}, Total Error: {total_error/10000:.4f}")

# Step 5: Predict missing values for recommendations
predicted_matrix = np.dot(user_factors, item_factors.T)
print("\nPredicted User-Item Matrix:")
print(predicted_matrix)

# Step 6: Recommend items for a specific user
user_id = 0  # Example user
# Recommend top 2 items that user hasn't interacted with
user_ratings = user_item_matrix[user_id]
predicted_ratings = predicted_matrix[user_id]
recommendations = np.argsort(predicted_ratings - user_ratings)  # Sort by predicted rating

# Only recommend items not yet interacted with
recommended_items = [i for i in recommendations if user_ratings[i] == 0][:2]
print(f"\nTop recommendations for User {user_id} (Age {users[user_id]}): Clothing Item IDs {clothing_items[recommended_items]}")


Epoch 0, Total Error: 3.3029
Epoch 10, Total Error: 0.8344
Epoch 20, Total Error: 0.7806
Epoch 30, Total Error: 0.7582
Epoch 40, Total Error: 0.7451
Epoch 50, Total Error: 0.7367
Epoch 60, Total Error: 0.7311
Epoch 70, Total Error: 0.7272
Epoch 80, Total Error: 0.7242
Epoch 90, Total Error: 0.7220

Predicted User-Item Matrix:
[[4.20630769 4.65338807 4.38261797 ... 4.75983752 3.84954596 3.91971399]
 [4.13857474 4.63618441 4.81592329 ... 4.45747263 4.03503162 3.82858239]
 [3.88145727 4.4048747  5.01182334 ... 3.95875665 4.02750946 3.56319829]
 ...
 [4.56444603 5.03973472 4.66972739 ... 5.20364816 4.13505063 4.25823435]
 [4.52785735 5.06512138 5.20649497 ... 4.90471494 4.38391605 4.19217701]
 [4.82638625 5.32730726 4.92336779 ... 5.50869483 4.36530538 4.5033993 ]]

Top recommendations for User 0 (Age 60): Clothing Item IDs [1176  118]


In [17]:
import pandas as pd
import numpy as np
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import mean_squared_error

# Assuming 'data' is your original dataframe
df = data.copy()

# Step 1: Adjust index to start from 2 and create the 'user_id' column
df['user_id'] = df.index  # Use the current index as the user_id
df['user_id'] = df['user_id'] + 2  # Adjust index so it starts from 2

# Step 2: Create the user-item interaction matrix (pivot table)
# Using 'user_id' for the index and 'Clothing ID' for the columns
interaction_matrix = df.pivot_table(index='user_id', columns='Clothing ID', values='Rating', fill_value=0)

# Step 3: Apply SVD (Singular Value Decomposition) - Matrix factorization technique
svd = TruncatedSVD(n_components=2)  # Use 2 components, adjust for better performance if needed
latent_matrix = svd.fit_transform(interaction_matrix)

# Step 4: Reconstruct the approximation of the original matrix
approx_matrix = np.dot(latent_matrix, svd.components_)

# Step 5: Calculate RMSE (Root Mean Square Error) for prediction accuracy
# We need to compare the predicted values with actual ratings
# Flatten both the original and approximated matrices for comparison
original_matrix_flat = interaction_matrix.values.flatten()
approx_matrix_flat = approx_matrix.flatten()

# Mask the zero entries in the interaction matrix (because they represent missing values)
mask = original_matrix_flat != 0
rmse = np.sqrt(mean_squared_error(original_matrix_flat[mask], approx_matrix_flat[mask]))
print("Root Mean Square Error (RMSE):", rmse)

# Step 6: Making Recommendations for a specific user
user_id = 5  # Example user, can be any user_id from your dataset
if user_id not in interaction_matrix.index:
    print(f"User {user_id} not found.")
else:
    user_row_index = interaction_matrix.index.get_loc(user_id)  # Find the row index for the user

    # Predict the ratings for items the user hasn't rated yet
    unrated_items = interaction_matrix.iloc[user_row_index] == 0  # Find unrated items
    predicted_ratings = approx_matrix[user_row_index]

    # Get predictions for unrated items
    recommendations = []
    for i, unrated in enumerate(unrated_items):
        if unrated:  # If the item is unrated
            recommendations.append((interaction_matrix.columns[i], predicted_ratings[i]))

    # Sort by predicted rating and recommend the top items
    recommendations.sort(key=lambda x: x[1], reverse=True)
    top_recommendations = recommendations[:3]  # Top 3 recommendations
    print(f"Top Recommendations for User {user_id}: {top_recommendations}")


Root Mean Square Error (RMSE): 4.156795365039826
Top Recommendations for User 5: [(np.int64(1078), np.float64(2.847168166262219e-09)), (np.int64(868), np.float64(1.2776029522272406e-11)), (np.int64(1077), np.float64(6.03820272475959e-12))]
